極性分析に有用そうな素性を各自で設計し，学習データから素性を抽出せよ．素性としては，レビューからストップワードを除去し，各単語をステミング処理したものが最低限のベースラインとなるであろう．


ステミングではなくて、複数形を単数形にするたどの処理の方がベター

In [1]:
import warnings
from nltk.corpus import stopwords
import pandas as pd
import stanfordnlp

In [2]:
stop_words = stopwords.words('english')

In [4]:
# プロセッサをデフォルトの全指定にするおｔ遅かったので最低限に絞る
# https://stanfordnlp.github.io/stanfordnlp/processors.html
nlp = stanfordnlp.Pipeline(processors='tokenize,pos,lemma')

Use device: cpu
---
Loading: tokenize
With settings: 
{'model_path': '/home/i348221/stanfordnlp_resources/en_ewt_models/en_ewt_tokenizer.pt', 'lang': 'en', 'shorthand': 'en_ewt', 'mode': 'predict'}
---
Loading: pos
With settings: 
{'model_path': '/home/i348221/stanfordnlp_resources/en_ewt_models/en_ewt_tagger.pt', 'pretrain_path': '/home/i348221/stanfordnlp_resources/en_ewt_models/en_ewt.pretrain.pt', 'lang': 'en', 'shorthand': 'en_ewt', 'mode': 'predict'}
---
Loading: lemma
With settings: 
{'model_path': '/home/i348221/stanfordnlp_resources/en_ewt_models/en_ewt_lemmatizer.pt', 'lang': 'en', 'shorthand': 'en_ewt', 'mode': 'predict'}
Building an attentional Seq2Seq model...
Using a Bi-LSTM encoder
Using soft attention for LSTM.
Finetune all embeddings.
[Running seq2seq lemmatizer with edit classifier]
Done loading processors!
---


In [5]:
# tag は Universal POS tags に準拠していそう
# https://universaldependencies.org/u/pos/
EXC_POS = {'PUNCT',   # 句読点
           'X',       # その他
           'SYM',     # 記号
           'PART',    # 助詞('sなど)
           'NUM'}     # 番号

In [6]:
# 3件確認のみ
with open('./sentiment.txt') as file:
    for i, line in enumerate(file):
        doc = nlp(line)
        print(i, line)
        for sentence in doc.sentences:
            for word in sentence.words:
                if   word.upos not in EXC_POS\
                 and word.lemma not in stop_words:
                    print(word.text, word.lemma, word.upos)
        if i == 3:
            break

0 -1 it's not original enough .

original original ADJ
enough enough ADJ
1 -1 gibney and jarecki just want to string the bastard up .

gibney gibney NOUN
jarecki jarecki PROPN
want want VERB
string string VERB
bastard bastard NOUN
2 -1 an overstylized , puréed mélange of sex , psychology , drugs and philosophy . sometimes entertaining , sometimes indulgent -- but never less than pure wankery .

overstylized overstylize VERB
puréed pur VERB
mélange mélange NOUN
sex sex NOUN
psychology psychology NOUN
drugs drug NOUN
philosophy philosophy NOUN
sometimes sometimes ADV
entertaining entertaining ADJ
sometimes sometimes ADV
indulgent indulgent ADJ
never never ADV
less less ADJ
pure pure ADJ
wankery wankery NOUN
3 +1 woo's fights have a distinct flair . his warriors collide in balletic explosion that implies an underlying order throughout the chaos .

woo woo NOUN
fights fight NOUN
distinct distinct ADJ
flair flair NOUN
warriors warrior NOUN
collide collide VERB
balletic balletic ADJ
explosio

In [7]:
# 警告非表示
warnings.simplefilter('ignore', UserWarning)

lemma = []
pos = []

pd_words = {'lemma':[], 'pos':[]}
with open('./sentiment.txt') as file:
    for i, line in enumerate(file):
        print("\r{0}".format(i), end="")
        doc = nlp(line)
        for sentence in doc.sentences:
            for word in sentence.words:
                if   word.upos not in EXC_POS\
                 and word.lemma not in stop_words:
                    lemma.append(word.lemma)
                    pos.append(word.pos)

10661

In [ ]:
                    pd_words['lemma'].append(word.lemma)
                    pd_words['pos'].append(word.upos)
df_words = pd.DataFrame(pd_words)

In [ ]:
df_words.head()
df_words.tail()

In [ ]:
lemma_count = df_words['lemma'].value_counts()
print(lemma_count)

In [ ]:
pos_count = df_words['pos'].value_counts()
print(pos_count)